# HOTELS - Analyse Verbatim-clients déposés sur Trip Advisor 

(1) Préparation du répertoire et des principaux packages

In [1]:
import os
os.chdir("C:/Disney")

In [2]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import gensim
print(gensim.__version__)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\chrys\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


4.1.2


# I. Analyses préparatoires avec les avis sur l'hôtel Marvel

**Objectif : test& learn** -> réaliser les premières analyses sur un petit fichier comprenant uniquement des avis récents afin d'enchaîner sur un modèle de généralisation plus robuste

**(1) Premières visualisations du fichier**
préalablement traduit et nettoyé avec les dictionnaires stop of words de la librairie ntlk (cf.notre fonction de nettoyage)

In [3]:
marvel=pd.read_csv("hotel_marvel_clean.csv", sep=",")
marvel.info( ) #439 avis

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   titre_commentaire  440 non-null    object
 1   commentaire        440 non-null    object
 2   Date               440 non-null    object
 3   Ville              440 non-null    object
 4   Pays               440 non-null    object
 5   Note               440 non-null    int64 
 6   Photo              440 non-null    object
 7   langue             440 non-null    object
dtypes: int64(1), object(7)
memory usage: 27.6+ KB


In [4]:
#visualisation des premières lignes
marvel.head()

,titre_commentaire,commentaire,Date,Ville,Pays,Note,Photo,langue
0,"['super', 'hôtel', 'thème', 'marvel', 'disneyl...","['super', 'hôtel', 'thème', 'marvel', 'décors'...",déc 2022,Valenciennes,France,5,yes,fr
1,"['attendais', 'mieux']","['peu', 'déçue', 'hôtel', 'très', 'belle', 'ch...",nov 2022,None,None,4,no,fr
2,['ravie'],"['avon', 'vraiment', 'apprécié', 'nuit', 'cet'...",nov 2022,None,None,4,no,fr
3,"['avis', 'mitigé']","['bel', 'hôtel', 'neuf', 'magique', 'yeux', 'e...",nov 2022,Calvi,France,3,no,fr
4,['fabuleux'],"['superbe', 'accueil', 'castmembers', 'récepti...",nov 2022,Toulouse,France,5,yes,fr


In [5]:
#Le fichier comprend des avis datés entre juin 2021 et décembre 2022
marvel['Date'].tail(1)

439    juin 2022
Name: Date, dtype: object

In [6]:
#Comptage des mots pour se donner une première idée des thèmes et d'un nettoyage complémentaire éventuel à opérer
import re
import collections

In [7]:
# titre 
t=marvel['titre_commentaire']
t.head()

0    ['super', 'hôtel', 'thème', 'marvel', 'disneyl...
1                               ['attendais', 'mieux']
2                                            ['ravie']
3                                   ['avis', 'mitigé']
4                                         ['fabuleux']
Name: titre_commentaire, dtype: object

In [8]:
#Comptage des mots dans le titre # Mots vides potentiellement restant (à voir) : hôtel, marvel,disneyland
#les mots de 3 lettres ou moins peuvent en général faire l'objet de suppression mais paraissent pertinents ici ex."fun, bel"
bagsofwords=[collections.Counter(re.findall(r'\w+',txt)) for txt in t]
sumbags = sum(bagsofwords, collections.Counter())
sumbags

Counter({'super': 30,
         'hôtel': 101,
         'thème': 3,
         'marvel': 37,
         'disneyland': 13,
         'paris': 9,
         'attendais': 4,
         'mieux': 5,
         'ravie': 2,
         'avis': 5,
         'mitigé': 4,
         'fabuleux': 4,
         'critique': 1,
         'honnête': 1,
         'ceux': 1,
         'normes': 2,
         'élevées': 1,
         'bon': 20,
         'génial': 2,
         'new': 8,
         'york': 8,
         'merveille': 5,
         'très': 47,
         'bien': 12,
         'situé': 4,
         'trop': 6,
         'cher': 10,
         'personnel': 9,
         'adorable': 1,
         'excellente': 1,
         'option': 1,
         'rester': 1,
         'pendant': 1,
         'vue': 3,
         'parc': 2,
         'vaut': 5,
         'peine': 2,
         'être': 3,
         'répété': 1,
         'héros': 3,
         'mauvais': 4,
         'service': 17,
         'définition': 1,
         'luxe': 3,
         'rêves': 1,
         

In [9]:
# commentaire
tx=marvel['commentaire']
tx.head()

0    ['super', 'hôtel', 'thème', 'marvel', 'décors'...
1    ['peu', 'déçue', 'hôtel', 'très', 'belle', 'ch...
2    ['avon', 'vraiment', 'apprécié', 'nuit', 'cet'...
3    ['bel', 'hôtel', 'neuf', 'magique', 'yeux', 'e...
4    ['superbe', 'accueil', 'castmembers', 'récepti...
Name: commentaire, dtype: object

In [10]:
#Comptage des mots dans les commentaires # Mots vides ? pertinence possible (à voir): hôtel,avis.meme remarque sur mots courts
#ex.lit -> pertinent de conserver mots d'au moins 3 lettres
bagsofwords=[collections.Counter(re.findall(r'\w+',txt)) for txt in tx]
sumbags = sum(bagsofwords, collections.Counter())
sumbags

Counter({'super': 125,
         'hôtel': 670,
         'thème': 59,
         'marvel': 231,
         'décors': 8,
         'couper': 4,
         'souffle': 3,
         'chambres': 206,
         'spacieuses': 30,
         'bien': 187,
         'équipée': 6,
         'machine': 24,
         'café': 39,
         'bouilloire': 12,
         'thé': 16,
         'petite': 44,
         'attention': 33,
         'souvenir': 11,
         'personnel': 228,
         'accueil': 64,
         'boutique': 22,
         'bagagerie': 16,
         'top': 62,
         'très': 528,
         'serviable': 43,
         'aimable': 6,
         'petit': 166,
         'soin': 12,
         'peu': 94,
         'déçue': 7,
         'belle': 66,
         'literie': 32,
         'déjeuner': 121,
         'classique': 6,
         'côté': 18,
         'surfait': 3,
         'dépaysée': 2,
         'charmée': 2,
         'comme': 86,
         'disneyland': 66,
         'plus': 189,
         'aucune': 18,
         'réducti

**(2) Word2Vec avec Gensim**

Pour pouvoir réaliser la vectorisation avec Word2Vec, il sebmble falloir répliquer le processus de tokenisation dans le notebook (sinon, ça ne marche pas, les vecteurs seront réalisés sur les lettres). 
Ne trouve pas l'import à réaliser pour faire identifier les tokens comme des tokens

In [18]:
tx[0]

"['super', 'hôtel', 'thème', 'marvel', 'décors', 'couper', 'souffle', 'chambres', 'spacieuses', 'bien', 'équipée', 'machine', 'café', 'bouilloire', 'thé', 'petite', 'attention', 'souvenir', 'personnel', 'accueil', 'boutique', 'bagagerie', 'top', 'très', 'serviable', 'aimable', 'petit', 'soin']"

In [20]:
print(type(tx))

<class 'pandas.core.series.Series'>


In [19]:
#suppresion de qq gros mots vides complementaires
mots_vides ="marvel","disney"
corpus=[[mot for mot in doc if not (mot in mots_vides)] for doc in tx]
print(corpus[0])

['[', "'", 's', 'u', 'p', 'e', 'r', "'", ',', ' ', "'", 'h', 'ô', 't', 'e', 'l', "'", ',', ' ', "'", 't', 'h', 'è', 'm', 'e', "'", ',', ' ', "'", 'm', 'a', 'r', 'v', 'e', 'l', "'", ',', ' ', "'", 'd', 'é', 'c', 'o', 'r', 's', "'", ',', ' ', "'", 'c', 'o', 'u', 'p', 'e', 'r', "'", ',', ' ', "'", 's', 'o', 'u', 'f', 'f', 'l', 'e', "'", ',', ' ', "'", 'c', 'h', 'a', 'm', 'b', 'r', 'e', 's', "'", ',', ' ', "'", 's', 'p', 'a', 'c', 'i', 'e', 'u', 's', 'e', 's', "'", ',', ' ', "'", 'b', 'i', 'e', 'n', "'", ',', ' ', "'", 'é', 'q', 'u', 'i', 'p', 'é', 'e', "'", ',', ' ', "'", 'm', 'a', 'c', 'h', 'i', 'n', 'e', "'", ',', ' ', "'", 'c', 'a', 'f', 'é', "'", ',', ' ', "'", 'b', 'o', 'u', 'i', 'l', 'l', 'o', 'i', 'r', 'e', "'", ',', ' ', "'", 't', 'h', 'é', "'", ',', ' ', "'", 'p', 'e', 't', 'i', 't', 'e', "'", ',', ' ', "'", 'a', 't', 't', 'e', 'n', 't', 'i', 'o', 'n', "'", ',', ' ', "'", 's', 'o', 'u', 'v', 'e', 'n', 'i', 'r', "'", ',', ' ', "'", 'p', 'e', 'r', 's', 'o', 'n', 'n', 'e', 'l', "'",

In [22]:
corpus =tx.tolist()
print(type(corpus))
print(corpus[0])

<class 'list'>
['super', 'hôtel', 'thème', 'marvel', 'décors', 'couper', 'souffle', 'chambres', 'spacieuses', 'bien', 'équipée', 'machine', 'café', 'bouilloire', 'thé', 'petite', 'attention', 'souvenir', 'personnel', 'accueil', 'boutique', 'bagagerie', 'top', 'très', 'serviable', 'aimable', 'petit', 'soin']


In [23]:
#suppresion de qq gros mots vides complementaires
mots_vides ="marvel","disney"
corpus2=[[mot for mot in doc if not (mot in mots_vides)] for doc in corpus]
print(corpus2[0])

['[', "'", 's', 'u', 'p', 'e', 'r', "'", ',', ' ', "'", 'h', 'ô', 't', 'e', 'l', "'", ',', ' ', "'", 't', 'h', 'è', 'm', 'e', "'", ',', ' ', "'", 'm', 'a', 'r', 'v', 'e', 'l', "'", ',', ' ', "'", 'd', 'é', 'c', 'o', 'r', 's', "'", ',', ' ', "'", 'c', 'o', 'u', 'p', 'e', 'r', "'", ',', ' ', "'", 's', 'o', 'u', 'f', 'f', 'l', 'e', "'", ',', ' ', "'", 'c', 'h', 'a', 'm', 'b', 'r', 'e', 's', "'", ',', ' ', "'", 's', 'p', 'a', 'c', 'i', 'e', 'u', 's', 'e', 's', "'", ',', ' ', "'", 'b', 'i', 'e', 'n', "'", ',', ' ', "'", 'é', 'q', 'u', 'i', 'p', 'é', 'e', "'", ',', ' ', "'", 'm', 'a', 'c', 'h', 'i', 'n', 'e', "'", ',', ' ', "'", 'c', 'a', 'f', 'é', "'", ',', ' ', "'", 'b', 'o', 'u', 'i', 'l', 'l', 'o', 'i', 'r', 'e', "'", ',', ' ', "'", 't', 'h', 'é', "'", ',', ' ', "'", 'p', 'e', 't', 'i', 't', 'e', "'", ',', ' ', "'", 'a', 't', 't', 'e', 'n', 't', 'i', 'o', 'n', "'", ',', ' ', "'", 's', 'o', 'u', 'v', 'e', 'n', 'i', 'r', "'", ',', ' ', "'", 'p', 'e', 'r', 's', 'o', 'n', 'n', 'e', 'l', "'",

**On commence avec un vecteur à 2 dimensions pour réaliser de premiers graphiques et un voisinage de 5 termes**

In [26]:
#word2vec
from gensim.models import Word2Vec
modele = Word2Vec(corpus,vector_size=2,window=5) #size = 2 ; 2 axes #window= 5
type(modele)

gensim.models.word2vec.Word2Vec

In [27]:
#word vectors (wv) = transcription en vecteurs de la liste des termes
words= modele.wv
print(type(words))

<class 'gensim.models.keyedvectors.KeyedVectors'>


In [ ]:
#dir(words)

In [28]:
words.key_to_index

{"'": 0,
 ' ': 1,
 ',': 2,
 'e': 3,
 'r': 4,
 't': 5,
 'a': 6,
 'i': 7,
 's': 8,
 'n': 9,
 'o': 10,
 'u': 11,
 'l': 12,
 'c': 13,
 'p': 14,
 'm': 15,
 'é': 16,
 'd': 17,
 'b': 18,
 'v': 19,
 'h': 20,
 'f': 21,
 'g': 22,
 'è': 23,
 'q': 24,
 'j': 25,
 'ô': 26,
 'x': 27,
 'y': 28,
 '[': 29,
 ']': 30,
 'z': 31,
 'k': 32,
 'ê': 33,
 '…': 34,
 'w': 35,
 'ç': 36,
 'î': 37,
 'û': 38,
 'à': 39,
 'â': 40,
 'œ': 41,
 'ë': 42,
 'á': 43,
 'ï': 44}

In [ ]:
#les termes les plus proches de
words.most_similar(" ",topn=5)

#les plus similaires de la conjonction et
words.most_similar(positive=[" "," "],topn=5)

#proche de "" mais loin de ""
print(words.most_similar(positive=[" "],negative=[" "],topn=5))

#chercher l'intrus
liste = [' ',' ']
words.doesnt_match(liste)